<a href="https://colab.research.google.com/github/harini-si/saidl-assignment/blob/main/semi_supervised_saidl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following notebook contains code for supervised and semi supervised part of computer vision assignment.
The architecture used is a resnet 18 except the first layer dimensions have been modified to fit the input image size.
Hyperparameters for supervised training:
1. number of epochs- 16
2. batch size-32
3. learning rate-0.001

(The results in the notebook are not the optimum results but close to it)

The results in the notebook show a difference of only 1 per cent between supervised and semi supervised methods

Accuracy for supervised learning

train accuracy- 85%

test accuracy- 61%

Hyperparamaters for semi supervised learning
1. number of epochs- 15
2. batch size-64
3. learning rate-0.001
4. threshold value- 0.99

Accuracy for semi-supervised learning

train accuracy-93 %

test accuracy-64 %

The model is trained from the beginning after adding the labels i.e. weights from earlier training are not retained.

In [ ]:
#imports
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets
import torch.optim as optim
from PIL import Image
import numpy as np

In [ ]:
#loading the data
train_data=datasets.STL10(root="./data",split='train',download=True,transform=ToTensor())
test_data=datasets.STL10(root="./data",split='test',download=True,transform=ToTensor())
train_loader=DataLoader(train_data,shuffle=True,batch_size=32)
test_loader=DataLoader(test_data,shuffle=True,batch_size=32)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [ ]:
import torch.backends.cudnn as cudnn
device = 'cuda'


In [ ]:
torch.cuda.empty_cache()

In [ ]:
#defining the network
class block(nn.Module):
  def __init__(self,in_channels,out_channels,stride=1):
    super(block,self).__init__()
    self.flag = False
    self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1)
    self.bn1=nn.BatchNorm2d(out_channels)
    self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1)
    self.bn2=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU()
    if stride != 1 or in_channels != out_channels:
      self.flag = True
      self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
    else:
      self.shortcut = nn.Sequential()
  def forward(self, x):
    out = self.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    if self.flag:
      x = self.shortcut(x)
    out += x
    out = self.relu(out)
    return out
    
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet,self).__init__()
    self.conv1= nn.Conv2d(3,64,kernel_size= 3,stride=1,padding=1,bias=False)
   
    
    self.bn1=nn.BatchNorm2d(64)
    self.conv2a=block(64,64)
    self.conv2b=block(64,64,1)
    self.conv3a=block(64,64)
    self.conv3b=block(64,128,2)
    
    self.conv4a=block(128,128)
    self.conv4b=block(128,256,2)
    
    
    self.conv5a=block(256,256)
    self.conv5b=block(256,512,2)
    
    self.relu = nn.ReLU()
    self.dropout=nn.Dropout2d(0.5)
    self.avg_pool=nn.AvgPool2d(kernel_size=4)
    self.linear = nn.Linear(512 , 10)
  def forward(self,x):
    x=self.relu(self.bn1(self.conv1(x)))
    
 
    x=self.conv2a(x)
    
    x=self.conv2b(x)
    x=F.max_pool2d(x,2,2)
    x=self.conv3a(x)
   
    x=self.conv3b(x)
    
    
    
    x=self.conv4a(x)
    
    x=self.conv4b(x)
   
    x=self.conv5a(x)
   
    
    x=self.conv5b(x)


    x=self.avg_pool(x)
    
    x=x.view(x.size(0), -1)
    x=self.dropout(x)
    x=self.linear(x)
   
    return x

In [ ]:
#hyperparameters
model=ResNet().to('cuda')
learning_rate=0.001

num_epochs=16

In [ ]:
#loss and optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999),weight_decay=2e-4, eps=1e-08, amsgrad=False)


In [ ]:
#def test loop and supervised training
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X=X.to('cuda')
            y=y.to('cuda')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    Accuracy=100*correct
    return Accuracy
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
   
    correct=0
    total=0
    for i, (images, labels) in enumerate(train_loader):
      
        images = images.to('cuda')
        labels = labels.to('cuda')

        
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
        
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
      

        if (i+1) % 20 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      
    
    accu= 100 * correct / len(train_data)
    test_accu=test_loop(test_loader,model)
    print(f'Epoch [{epoch+1}/{num_epochs}], train_accuracy: {accu:.4f},test_accuracy: {test_accu:.4f} %')   


Epoch [1/16], Step [20/157], Loss: 2.0694
Epoch [1/16], Step [40/157], Loss: 1.9475
Epoch [1/16], Step [60/157], Loss: 1.6442
Epoch [1/16], Step [80/157], Loss: 1.7202
Epoch [1/16], Step [100/157], Loss: 1.7411
Epoch [1/16], Step [120/157], Loss: 1.8664
Epoch [1/16], Step [140/157], Loss: 1.7924
Epoch [1/16], train_accuracy: 28.0200,test_accuracy: 32.6250 %
Epoch [2/16], Step [20/157], Loss: 1.9640
Epoch [2/16], Step [40/157], Loss: 1.8036
Epoch [2/16], Step [60/157], Loss: 1.8491
Epoch [2/16], Step [80/157], Loss: 1.7589
Epoch [2/16], Step [100/157], Loss: 1.6170
Epoch [2/16], Step [120/157], Loss: 1.6755
Epoch [2/16], Step [140/157], Loss: 1.7245
Epoch [2/16], train_accuracy: 28.9600,test_accuracy: 33.0375 %
Epoch [3/16], Step [20/157], Loss: 1.6037
Epoch [3/16], Step [40/157], Loss: 1.5868
Epoch [3/16], Step [60/157], Loss: 1.3383
Epoch [3/16], Step [80/157], Loss: 1.7759
Epoch [3/16], Step [100/157], Loss: 1.7721
Epoch [3/16], Step [120/157], Loss: 1.4775
Epoch [3/16], Step [140/15

In [ ]:

PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
unlabeled=datasets.STL10(root="./data",split='unlabeled',download=True,transform=ToTensor())

unlabeled1=torch.utils.data.Subset(unlabeled,range(0,int (len(unlabeled)/2), 2))
unlabelled_loader1=DataLoader(unlabeled1,shuffle=True,batch_size=32)

Files already downloaded and verified


In [ ]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
#threshold for p labels
num_epochs1=10
threshold=0.999

In [ ]:
def generate_labels(unlabelled_loader1,threshold):
  #model.eval()
  x=0
  plabels=[]
  image=[]
  with torch.no_grad():
    for i, (images,labels) in enumerate (unlabelled_loader1):
      images=images.to('cuda')
      labels=labels.to('cuda')
    
      outputs=model(images)
      outputs=F.softmax(outputs,dim=1)
      if((outputs.max()>threshold) ):
      
        predicted=outputs.max(1)[1]
        plabels.append(predicted)
        image.append(images)
      
      
      
    return plabels,image
pl,imgs=generate_labels(unlabelled_loader1,threshold)
print(len(imgs))

386


In [ ]:
dataTensor = torch.cat(pl)
print(dataTensor)
dataTensor.to('cuda')
imgTensor=torch.cat(imgs)
print(len(imgTensor))
imgTensor.to('cuda')
i=[imgTensor]
p=[dataTensor]

tensor([5, 7, 1,  ..., 8, 7, 5], device='cuda:0')
12352


In [ ]:
class MyDataset(Dataset):
    def __init__(self, data,targets, transform=None):
        self.data = i[0]
        self.targets = p[0]
       
        
    def __getitem__(self, index):
        x = self.data[index]
        y=self.targets[index]
     

     
        
        return x,y
    
    def __len__(self):
        return len(self.data)



transform = transforms.Compose([transforms.Resize(96), transforms.ToTensor()])
dataset = MyDataset(i,p )
print(dataset[0][0].shape)



torch.Size([3, 96, 96])


In [ ]:
x=[]
y=[]
for i,(images,labels) in enumerate(train_loader):
  images=images.to(device)
  labels=labels.to(device)
  x.append(images)
  y.append(labels)

In [ ]:
train_x=torch.cat(x)
train_y=torch.cat(y)
type(train_y)

torch.Tensor

In [ ]:
class tDataset(Dataset):
    def __init__(self, data,targets, transform=None):
        self.data = train_x
        self.targets = train_y
        print(len(self.data))
        
        self.transform = transform
        
    def __getitem__(self, i):
        x = self.data[i]
        y=self.targets[i]
        return x,y
    
    def __len__(self):
        return len(self.data)
data = tDataset(train_x,train_y)
len(data)


5000


5000

In [ ]:
concatDataset=torch.utils.data.ConcatDataset([dataset,data])
loader = DataLoader(concatDataset, batch_size=64,shuffle=True)
len(concatDataset)

17352

In [ ]:
model=ResNet().to(device)

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=2e-4, eps=1e-08, amsgrad=False)

In [ ]:
n_total_steps = len(loader)
for epoch in range(num_epochs1):
   
    correct=0
    total=0
    for i, (images, labels) in enumerate(loader):
      
    
        images = images.to(device)
        labels = labels.to(device,dtype=torch.int64)
      

        
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
        
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
      

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs1}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      
    
    accu= 100 * correct / len(concatDataset)
    test_accu=test_loop(test_loader,model)
    print(f'Epoch [{epoch+1}/{num_epochs1}], train_accuracy: {accu:.4f},test_accuracy: {test_accu:.4f} %')   

Epoch [1/15], Step [100/272], Loss: 1.5777
Epoch [1/15], Step [200/272], Loss: 1.2247
Epoch [1/15], train_accuracy: 39.4306,test_accuracy: 42.0375 %
Epoch [2/15], Step [100/272], Loss: 1.9796
Epoch [2/15], Step [200/272], Loss: 1.8351
Epoch [2/15], train_accuracy: 25.3688,test_accuracy: 32.0125 %
Epoch [3/15], Step [100/272], Loss: 1.6296
Epoch [3/15], Step [200/272], Loss: 1.3386
Epoch [3/15], train_accuracy: 37.3156,test_accuracy: 36.8875 %
Epoch [4/15], Step [100/272], Loss: 1.3049
Epoch [4/15], Step [200/272], Loss: 1.2031
Epoch [4/15], train_accuracy: 47.7006,test_accuracy: 34.6750 %
Epoch [5/15], Step [100/272], Loss: 1.3889
Epoch [5/15], Step [200/272], Loss: 1.2523
Epoch [5/15], train_accuracy: 53.7229,test_accuracy: 40.2375 %
Epoch [6/15], Step [100/272], Loss: 1.1260
Epoch [6/15], Step [200/272], Loss: 1.0505
Epoch [6/15], train_accuracy: 58.4255,test_accuracy: 46.3750 %
Epoch [7/15], Step [100/272], Loss: 0.9465
Epoch [7/15], Step [200/272], Loss: 1.1145
Epoch [7/15], train_